# Pseudotemporal ordering

## Motivation

Single-cell sequencing assays provide high resolution measurements of biological tissues __<span style="color: red;">[CITE]</span>__. Consequently, such technologies can help decipher and understand cellular heterogeneity __<span style="color: red;">[CITE]</span>__ and the dynamics of a biological process __<span style="color: red;">[CITE]</span>__. Corresponding studies include quantifying cellular fates as well as identifying genes driving the process. However, as cells are destroyed when sequenced, it is impossible to track their development and, for example,  gene expression profile over time. Instead, the underlying dynamic process needs to be estimated from the measured snapshot data.

Even though samples are, traditionally, taken from a single experimental time point, a multitude of cell types can be observed. This diversity stems from the asynchronous nature of biological processes. As such, a range of the developmental process can be observed. Reconstructing the developmental landscape is the goal of the field coined trajectory inference (TI). This task is achieved by ordering the observed cellular states according to the developmental process. States are aligned along the developlemtal direction by mapping discrete annotations to a continuous domain - the so-called _pseudotime_.

A pseudotime ranks cells relative to each other according to their respective stage in the developmental process. Less mature cells are assigned small, mature cells large values. Studying a bone marrow sample for example, hematopoietic stem cells are assigned a low, and erythroid cells a high pseudotime. The assignment is, in case of single-cell RNA sequencing data, based on the transcriptomic profile of a cell. Additionally, the construction usually requires the specification of an initial cell where the overall process starts.

## CONTENT

## Key takeaways

## Quiz

## References

```{bibliography}
:filter: docname in docnames
```